In [ ]:
!pip install langchain-core langchain-openai langchain-exa
!pip install exa_py

### Exa Api

In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# Load the .env file
load_dotenv()

# Read the API key from the environment variable
oai_api_key = os.getenv("OPENAI_API_KEY")

exa_api_key = os.getenv("EXA_API_KEY")

openai = OpenAI()

### Search Request

In [4]:
from exa_py import Exa

exa = Exa(api_key=exa_api_key)
query = "latest ai cognitive architecture systems"

searches = exa.search(query,
    num_results=2,
    # include_domains=["nytimes.com", "wsj.com"],
    # exclude_domains=["reddit.com"],
    # start_crawl_date = "2021-06-12",
    # end_crawl_date = "2021-06-12",
    # start_published_date="2023-06-12"
    use_autoprompt=True,
    # type = 'keyword' # 'keyword' or 'neural
)

# Needs a relevance check with LLM + Puppetier

for search in searches.results:
    print(search.title, search.url)

Announcement https://soar.eecs.umich.edu/
Mind|Construct - ASTRID /Brain 5 http://www.mindconstruct.com//website/astrid


### Search + Contents

In [7]:
searches_contents = exa.search_and_contents(query,
    num_results=2,
    # include_domains=["nytimes.com", "wsj.com"],
    # exclude_domains=["reddit.com"],
    # start_crawl_date = "2021-06-12",
    # end_crawl_date = "2021-06-12",
    # start_published_date="2023-06-12"
    use_autoprompt=True,
    # type = 'keyword' # 'keyword' or 'neural
    text={"max_characters": 500},
)

for search in searches_contents.results:
    # print(search)
    print(search.title, search.url, "\n", search.text)

Cognitive Architecture https://cogarch.ict.usc.edu/ 
  
	The goal of this effort is to develop a sufficiently efficient, functionally elegant, generically cognitive, grand unified, cognitive architecture in support of virtual humans (and hopefully intelligent agents/robots – and even a new form of unified theory of human cognition – as well).
A cognitive architecture is a hypothesis about the fixed structures that provide a mind, whether in natural or artificial systems, and how they work together – in conjunction with knowledge and skills embodied
Announcement https://soar.eecs.umich.edu/ 
  
	What is Soar?
Soar is a general cognitive architecture for developing systems that exhibit intelligent behavior. Researchers all over the world, both from the fields of artificial intelligence and cognitive science, are using Soar for a variety of tasks. It has been in use since 1983, evolving through many different versions to where it is now Soar, Version 9.
We intend ultimately to enable the 

### GPT Summary of Contents

In [3]:
import textwrap

SYSTEM_MESSAGE = "You are a helpful assistant that briefly summarizes the content of a webpage. Summarize the users input."

def summarise_gpt(text):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": SYSTEM_MESSAGE},
            {"role": "user", "content": text},
        ],
    )
    return response.choices[0].message.content

for search in searches_contents.results:
    print(f"SUMMARY for {search.url}:")
    print(search.title)
    print(textwrap.fill(summarise_gpt(search.text), 100))

NameError: name 'searches_contents' is not defined

### Similarity Branching + Contents

In [8]:
for search in searches.results:
    similars = exa.find_similar(search.url, num_results=2)
    for similar in similars.results:

        contents = exa.get_contents([similar.id])
        for content in contents.results:
            print("Title:", content.title)
            print("URL:", content.url)
            print("Text:", content.text)

Title: Soar (cognitive architecture) - Wikipedia
URL: https://en.wikipedia.org/wiki/Soar_(cognitive_architecture)
Text:    Soar  [1]  is a cognitive architecture, [2]  originally created by John Laird, Allen Newell, and Paul Rosenbloom at Carnegie Mellon University. (Rosenbloom continued to serve as co-principal investigator after moving to Stanford University, then to the University of Southern California's Information Sciences Institute.) It is now maintained and developed by John Laird's research group at the University of Michigan.
 The goal of the Soar project is to develop the fixed computational building blocks necessary for general intelligent agents – agents that can perform a wide range of tasks and encode, use, and learn all types of knowledge to realize the full range of cognitive capabilities found in humans, such as decision making, problem solving, planning, and natural-language understanding. It is both a theory of what cognition is and a computational implementation of

### Simple RAG Example

In [ ]:
questions = [
    "What are the latest advancements in cognitive architecture AI systems?",
    "Propose a project plan for a cognitive architecture AI systems based on latest innovations and research.",
]

# Parameters for our Highlights search
highlights_options  = {
    "num_sentences": 7, # how long our highlights should be
    "highlights_per_url": 1, # just get the best highlight for each URL
    "query": "ai cognitive architecture"
}

info_for_llm = []
for question in questions:
    search_response = exa.search_and_contents(question, highlights=highlights_options, num_results=3, use_autoprompt=True)
    info = [sr.highlights[0] for sr in search_response.results]
    info_for_llm.append(info)


# ------- BUILT WEB CRAWLING KNOWLEDGE BASE as info_for_llm
# invisible URLS, cant pass them back to user with this approach


responses = []
for question, info in zip(questions, info_for_llm):
  system_prompt = "You always respond in Markdown format. Read the provided contexts and, if relevant, use them to answer the user's question."
  user_prompt = f"""Sources: {info}
  
  Question: {question}"""

  completion = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": user_prompt},
    ]
  )
  response = f"""
  Question: {question}
  Answer: {completion.choices[0].message.content}
  """
  responses.append(response)

for response in responses:
    print(response)
    


### LANGCHAIN

In [16]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

from langchain_exa import ExaSearchRetriever, TextContentsOptions
from langchain_openai import ChatOpenAI

# retrieve 5 documents, with content truncated at 1000 characters
retriever = ExaSearchRetriever(
    k=5, text_contents_options=TextContentsOptions(max_length=1000)
)

# print(RunnableParallel({"context": retriever, "When is the best time to visit japan?": RunnablePassthrough()}))
    

prompt = PromptTemplate.from_template(
    """Answer the following query based on the following context:
query: {query}
<context>
{context}
</context"""
)

llm = ChatOpenAI(model="gpt-4-vision-preview", api_key=oai_api_key)

chain = (
    RunnableParallel({"context": retriever, "query": RunnablePassthrough()})
    | prompt
    | llm
)

response = chain.invoke("what are the latest ai cognition inspired architectures").content

print(response)

Based on the provided context, the latest AI cognition-inspired architectures include the Sigma architecture


---
---
### Web Access Agent w Langchain - FAIL

In [14]:
from exa_py import Exa
from langchain.agents import tool

exa = Exa(api_key=exa_api_key)


@tool
def search(query: str, include_domains=None, start_published_date=None):
    """Search for a webpage based on the query.
    Set the optional include_domains (list[str]) parameter to restrict the search to a list of domains.
    Set the optional start_published_date (str) parameter to restrict the search to documents published after the date (YYYY-MM-DD).
    """
    return exa.search_and_contents(
        f"{query}",
        use_autoprompt=True,
        num_results=5,
        include_domains=include_domains,
        start_published_date=start_published_date,
    )

tools = [search]

print(search("latest ai cognitive architecture systems").results[0].text)

 
 Welcome to AERA
The Autocatalytic Endogenous Reflective Architecture (AERA) is a GMI-aspiring (general machine intelligence) system 
under development at the Center for Analysis & Design of Intelligent Agents (CADIA) 
at Reykjavik University and the Icelandic Institute for Intelligent Machines in Iceland (IIIM).

  
  What is AERA? 
  How is AERA different from prior work? 
  How does AERA learn? 
 
 
 AERA is a cognitive architecture - and a blueprint - for constructing agents with high levels of operational autonomy, starting from only a small amount of designer-specified code – a seed. Using a value-driven dynamic priority scheduling to control the parallel execution of a vast number of lines of reasoning, the system accumulates increasingly useful models of its experience, resulting in recursive self-improvement that can be autonomously sustained after the machine leaves the lab, within the boundaries imposed by its designers. 
 AERA demonstrates domain-independent self-supervis

In [ ]:
from langchain.agents import AgentExecutor, OpenAIFunctionsAgent
from langchain.schema import SystemMessage
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4-vision-preview")

system_message = SystemMessage(
    content="You are a web researcher who answers user questions by looking up information on the internet and retrieving contents of helpful documents. Cite your sources."
)

agent_prompt = OpenAIFunctionsAgent.create_prompt(system_message)
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.run(
    "Summarize for me an interesting article about AI from lesswrong.com published after October 2023."
)